In [141]:
#! ../../env/bin/python3

from bs4 import BeautifulSoup
import pandas as pd
import requests

DATADIR = '../../data/processed/housing-off-campus/'
zipcode = 92037
date = 113022

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    base_url = 'https://www.zillow.com/san-diego-ca-{}/rentals/'.format(zipcode)
    
    # initialize variables
    page_num = 1
    status = 200
    zillow_text = ''
    
    print('Scraping page {}'.format(page_num))
    
    while status == 200:
        # update every 5 pages
        if page_num % 5 == 0:
            print('Scraping page {}'.format(page_num))
        
        # add page number suffix to url
        if page_num != 1:
            url = base_url + str(page_num+1) + '_p/'
        else:
            url = base_url
            
        # get page    
        r = s.get(url, headers=req_headers)
        
        try:
            soup = BeautifulSoup(r.content, 'html.parser')
            
            # append text
            zillow_text += soup.get_text()
            
        except:
            print('Last page reached.\n Total pages scraped: {}'.format(page_num-1))
        
        # update status and page number                         
        status = r.status_code
        page_num += 1

Scraping page 1
Scraping page 5
Scraping page 10
Scraping page 15
Scraping page 20
Scraping page 25


In [146]:
zillow_text.split('resultsSort:')

['\n\nRental Listings in 92037 - 130 Rentals | Zillow\n\n\n\n\nWarningThis browser is no longer supported. Please switch to a supported browser or download one of our Mobile Apps.See Mobile AppsSkip main navigationSign in Join HomepageBuy Open Buy sub-menuChevron DownRent Open Rent sub-menuChevron DownSell Open Sell sub-menuChevron DownHome Loans Open Home Loans sub-menuChevron DownAgent finder Open Agent finder sub-menuChevron DownManage Rentals Open Manage Rentals sub-menuChevron DownAdvertise Help Sign in Join \n\n\nSubmit SearchSearchA magnifying glass For RentChevron DownPriceChevron DownBeds & BathsChevron DownHome TypeChevron DownMoreChevron DownSave search92037 Rental Listings130 ',
 ' DefaultChevron DownRegents La Jolla | 9253 Regents Rd, La Jolla, CA$3,220+ 1 bd$3,975+ 2 bds Updated todayTerraces at La Jolla Village | 8843 Villa La Jolla Dr, La Jolla, CA$3,395+ 2 bds Loading Loading...Ocean House on Prospect Apartment Homes | 400 Prospect St, La Jolla, CA$5,599+ 1 bd$5,999+ 2

In [154]:
import re
text = re.split(r'resultsSort: DefaultChevron Down|Left1234Page|Chevron', zillow_text)

listings = [x for x in text if '$' in x]

In [155]:
listings

['Regents La Jolla | 9253 Regents Rd, La Jolla, CA$3,220+ 1 bd$3,975+ 2 bds Updated todayTerraces at La Jolla Village | 8843 Villa La Jolla Dr, La Jolla, CA$3,395+ 2 bds Loading Loading...Ocean House on Prospect Apartment Homes | 400 Prospect St, La Jolla, CA$5,599+ 1 bd$5,999+ 2 bds Three Dimensional 3D TourSolazzo Apartments Homes | 8506 Villa La Jolla Dr, La Jolla, CA$2,960+ 1 bd$3,545+ 2 bds Three Dimensional 3D TourCasa Linda, 7623 Eads Ave, La Jolla, CA 92037$2,100+/mo1 bd1 ba400 sqft - Apartment for rent7147 Fay Ave, La Jolla, CA 92037$11,750/mo3 bds3.5 ba3,006 sqft - Apartment for rent4 hours ago7880 Avenida Kirjah, La Jolla, CA 92037$14,000/mo5 bds6 ba4,110 sqft - House for rent4 hours ago365 Bonair St, La Jolla, CA$2,950+ 1 bd$3,375+ 2 bds 2 days ago5559 Chelsea Ave, La Jolla, CA 92037$8,500/mo3 bds2.5 ba2,030 sqft - House for rent11 days ago',
 '6144 Castejon Dr, La Jolla, CA 92037$9,700/mo4 bds2 ba2,304 sqft - House for rent29 days ago5755 Waverly Ave, La Jolla, CA 92037$6,

In [87]:
import pandas as pd

zipcode = '92037'

with open("../data/raw/zillow/zip_{}.txt".format(zipcode), "r") as text_file:
    zillow_text = text_file.read().splitlines()

In [115]:
text = []
for x in zillow_text:
    if x.startswith('San Diego'):
        text.append(x.split('resultsSort: DefaultChevron Down')[1].split('Chevron Left1234Page')[0])

In [138]:
listings = []

for i in range(len(text)):
    for x in text[i].split('days ago'):
        if x is not '':
            listings.append(x.split('-')[0])

<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/wp/3f0wwp4n2yzg421rx1jgph980000gn/T/ipykernel_10505/3194578342.py:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if x is not '':


In [139]:
listings

['2075 Torrey Pines Rd, La Jolla, CA 92037$15,500/mo5 bds4 ba3,667 sqft ',
 '6144 Castejon Dr, La Jolla, CA 92037$9,700/mo4 bds2 ba2,304 sqft ',
 'Loading Loading...5755 Waverly Ave, La Jolla, CA 92037$6,750/mo3 bds2 ba1,360 sqft ',
 '7788 Sierra Mar Dr, La Jolla, CA 92037$7,500/mo3 bds2 ba1,890 sqft ',
 '5695 Soledad Mountain Rd, La Jolla, CA 92037$12,600/mo5 bds4.5 ba4,000 sqft ',
 '9683 Claiborne Sq, La Jolla, CA 92037$6,700/mo4 bds2.5 ba2,601 sqft ',
 '5385 La Jolla Mesa Dr, La Jolla, CA 92037$10,500/mo4 bds5 ba4,274 sqft ',
 '415 Gravilla St UNIT 3, La Jolla, CA 92037$3,000/mo1 bd1 ba400 sqft ',
 '7274 Via Capri, La Jolla, CA 92037$15,000/mo4 bds3.5 ba2,700 sqft ',
 '6436 Camino De La Costa, La Jolla, CA 92037$90,000/mo7 bds8.5 ba8,337 sqft ',
 '1110 W Muirlands Dr, La Jolla, CA 92037$18,000/mo3 bds4 ba3,340 sqft ',
 'Loading Loading...5582 Calle Miramar, La Jolla, CA 92037$19,995/mo4 bds3 ba4,046 sqft ',
 '369 Belvedere St, La Jolla, CA 92037$11,500/mo5 bds4 ba3,003 sqft ',
 '543

In [62]:
# initialize dataframe
df = pd.DataFrame(columns=['price', 'unit-type', 'beds', 'note-on-price', 'address'])

        
df['unit-type'] = [x.split(' ')[1] for x in zillow_text.split('$')[1:]]

In [83]:
zillow_text

"\n\nRental Listings in 92037 - 131 Rentals | Zillow\n\n\n\n\nWarningThis browser is no longer supported. Please switch to a supported browser or download one of our Mobile Apps.See Mobile AppsSkip main navigationSign in Join HomepageBuy Open Buy sub-menuChevron DownRent Open Rent sub-menuChevron DownSell Open Sell sub-menuChevron DownHome Loans Open Home Loans sub-menuChevron DownAgent finder Open Agent finder sub-menuChevron DownManage Rentals Open Manage Rentals sub-menuChevron DownAdvertise Help Sign in Join \n\n\n\n\n\n\n\n\nRental Listings in 92037 - 131 Rentals | Zillow\n\n\n\n\nWarningThis browser is no longer supported. Please switch to a supported browser or download one of our Mobile Apps.See Mobile AppsSkip main navigationSign in Join HomepageBuy Open Buy sub-menuChevron DownRent Open Rent sub-menuChevron DownSell Open Sell sub-menuChevron DownHome Loans Open Home Loans sub-menuChevron DownAgent finder Open Agent finder sub-menuChevron DownManage Rentals Open Manage Rentals

In [68]:
# PRICE
tmp = [x.split(' ')[0] for x in zillow_text.split('$')[1:]]
df['note-on-price'] = ['minimum price listed in range' if '+' in x else 'exact price' for x in tmp]

final = []
# data cleaning
for x in tmp:
    try:
        if '+' in x:
            final.append(int(x.split('+')[0].replace(',', '')))
        elif '/mo' in x:
            final.append(int(x.split('/mo')[0].replace(',', '')))
    except:
        print(x)
        break
    
df['price'] = final

In [82]:
df['unit-type'] = [x.split(' ')[1] for x in zillow_text.split('$')[1:]]

In [ ]:
df['unit-type'] = [x.split(' ')[1] for x in zillow_text.split('$')[1:]]
df['beds'] = [1 if x == 'Studio' else int(x) for x in df['unit-type']]
df['bedrooms'] = [0 if x == 'Studio' else int(x) for x in df['unit-type']]

# create price per bed column
df['price-per-bed'] = round(df['price'] / df['beds'], 2)

# get summary statistics
stats = pd.concat([df.describe().T, df.median(numeric_only=True)], axis=1).rename(columns={0: 'median'})

In [ ]:
# save to csv
df.to_csv('{}/current/zillow-{}-{}.csv'.format(DATADIR, zipcode, date), index=False)
stats.to_csv('{}/current/zillow-stats-{}-{}.csv'.format(DATADIR, zipcode, date), index=True)